In [ ]:
pip install python-dotenv

In [ ]:
pip install folium

## Login 

The login reads user credentials from a .env file then attempts to login in and get an authorised bearer token.

In [ ]:
from dotenv import load_dotenv
import requests
import pprint
import os

load_dotenv()

payload = {
    "email": os.getenv("API_EMAIL"),
    "password": os.getenv("API_PASSWORD")
}

response = requests.post("https://api-nwfp.rothamsted.ac.uk/api/auth/v1/login/", json=payload)

token = ""
headers = None

if response.status_code == 200:
    response_data = response.json()
    print("Login successful!")
    print("Response data:")
    pprint.pp(response_data)
    token = response_data.get("access")
    headers = {"Authorization": f"Bearer {token}"}
else:
    print("Login failed!")
    print("Status code:", response.status_code)
    print("Response:", response.text)

## Use the catchments end point to fetch catchment boundaries

- print the returned JSON response data to view

In [ ]:
import folium
import numpy as np
import json
from pandas import json_normalize
import pandas as pd
from collections import defaultdict

api_url = "https://api-nwfp.rothamsted.ac.uk/api/catchments/v1/getCatchments/"

params = {
    #"catchments": "catchment 5,met,catchment 6",
}

response = requests.get(api_url, headers=headers, params=params, verify=False)

if response.status_code == 200:
    print("API request successful!")
    catchment_data = response.json()
    pprint.pp(catchment_data)
else:
    print("Failed to login. Status code:", response.status_code)
    print("Response:", response.text)

## Render a clickable map

- Clicking on a map polygon should display a pop-up with basic information
- Polygons should be colour-coded depending on the farmlet (red, green or blue)
- Brown farmlet (housed) should be ignored by the map as they do not have associated polygons

In [ ]:
# Define colors for each unique farmlet name
farmlet_colors = defaultdict(lambda: None)
color_palette = ['#FF0000', '#964B00', '#0000FF', '#00FF00']  # Add more colors if needed

# Assign a color to each farmlet_name
for index, catchment in enumerate(catchment_data):
    farmlet_name = catchment['farmlet_name']
    if farmlet_colors[farmlet_name] is None:
        farmlet_colors[farmlet_name] = color_palette[index % len(color_palette)]

# Initialize the map centered on the first valid polygon's first point
initial_location = None
for catchment in catchment_data:
    if catchment.get('boundary') and catchment['boundary'][0]:
        initial_location = catchment['boundary'][0][0]
        break

if initial_location is None:
    raise ValueError("No valid boundary data found to initialize the map.")

m = folium.Map(location=initial_location, zoom_start=15)

# Iterate over the catchment data to add polygons to the map
for catchment in catchment_data:
    farmlet_name = catchment['farmlet_name']
    boundary = catchment.get('boundary')
    color = farmlet_colors[farmlet_name]

    # Skip if boundary data is missing or invalid
    if not boundary or not boundary[0]:
        print(f"Skipping catchment {catchment['catchment_id']} due to missing boundary data.")
        continue

    # Use the coordinates list directly for the polygon
    polygon_coords = boundary[0]

    # Format summary information for the popup
    popup_content = f"""
    <strong>Catchment ID:</strong> {catchment['catchment_id']}<br>
    <strong>Catchment Name:</strong> {catchment['catchment_name']}<br>
    <strong>Display Name:</strong> {catchment['displayname']}<br>
    <strong>Farmlet Name:</strong> {catchment['farmlet_name']}<br>
    <strong>Hydrological Catchment Area:</strong> {catchment['hydrologicalcatchmentarea']} ha<br>
    <strong>Fenced Catchment Area:</strong> {catchment['fencedcatchmentarea']} ha
    """
    popup = folium.Popup(popup_content, max_width=300)

    # Add the polygon to the map
    folium.Polygon(
        locations=polygon_coords,  # Directly use the coordinates list for the polygon
        color='black',
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        weight=2.5,
        popup=popup
    ).add_to(m)

# Display the map
m